In [2]:
#imports
import string
import pandas as pd
import pickle as pkl
from collections import Counter
from matplotlib import pyplot as plt

import numpy as np
from random import sample

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

import sys
sys.path.append('../../src')
import utils

2023-08-23 09:36:23.134149: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 09:36:23.730257: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import fasttext
import fasttext.util

# Download a pre-trained FastText model (change the parameters as needed)
model_path = fasttext.util.download_model('en', if_exists='ignore')  # 'en' for English

# Load the downloaded model
model = fasttext.load_model(model_path)

# Get embeddings for a word

def get_embeddings(word):
    return model.get_word_vector(word)

In [4]:
watches = pd.read_pickle('../../data/user-level-data')
videos = pd.read_pickle('../../data/videos_raw_metadata')

In [5]:
TOPICS = dict()
TAGS = dict()

In [6]:
all_topics = []
all_tags = []

for video in videos:
    video = videos[video]
    
    try:
        topics = video['topicDetails']['topicCategories']
        topics = [topic.split('/')[-1].lower() for topic in topics]
    except:
        topics = []

    all_topics += topics
    
    try:
        tags = video['snippet']['tags']
        tags = [tag.lower() for tag in tags]
    except:
        tags = []

    all_tags += tags

In [7]:
for topic in tqdm(all_topics):
    if topic not in TOPICS:
        TOPICS[topic] = get_embeddings(topic)

for tag in tqdm(all_tags):
    if tag not in TAGS:
        TAGS[tag] = get_embeddings(tag)

100%|██████████| 7679701/7679701 [00:29<00:00, 260078.65it/s]


In [38]:
def get_mean_tag_embedding(topics):
    embs = []
    for topic in topics:
        if topic in TAGS:
            embs.append(TAGS[topic])
    if len(embs) == 0:
        # return empty embedding
        return np.zeros(300)
    return np.mean(embs, axis=0)

def get_mean_topic_embedding(topics):
    embs = []
    for topic in topics:
        if topic in TOPICS:
            embs.append(TOPICS[topic])
    if len(embs) == 0:
        # return empty embedding
        return np.zeros(300)
    return np.mean(embs, axis=0)

In [40]:
topic_embeddings = dict()
tag_embeddings = dict()

for video_id in tqdm(videos):
    video = videos[video_id]
    
    try:
        topics = video['topicDetails']['topicCategories']
        topics = [topic.split('/')[-1].lower() for topic in topics]
    except:
        topics = []

    topic_embeddings[video_id] = get_mean_topic_embedding(topics)

    try:
        tags = video['snippet']['tags']
        tags = [tag.lower() for tag in tags]
    except:
        tags = []

    tag_embeddings[video_id] = get_mean_tag_embedding(topics)

100%|██████████| 524537/524537 [00:14<00:00, 36435.68it/s]


In [41]:
pkl.dump(topic_embeddings, open('../../data/embeddings/topic_embeddings.pkl', 'wb'))
pkl.dump(tag_embeddings, open('../../data/embeddings/tag_embeddings.pkl', 'wb'))

In [42]:
sample_videos = sample(list(videos.keys()), 100)

In [43]:
def get_tags(v):
    video = videos[v]
    try:
        tags = video['snippet']['tags']
        tags = [tag.lower() for tag in tags]
    except:
        tags = []

    return tags

In [52]:
tags = []
titles = []
for v in sample_videos:
    titles.append(videos[v]['snippet']['title'])
    tags.append(get_tags(v))

In [48]:
tag_embs = []
for tag in tags:
    tag_embs.append(get_mean_tag_embedding(tag))

tag_embs = np.array(tag_embs)

# cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

df = cosine_similarity(tag_embs)

In [55]:
df = pd.DataFrame(df, columns=titles, index=titles)

In [65]:
for v in sample_videos:
    t = videos[v]['snippet']['title']
    if t == title:
        print(get_tags(v))

['real face of', 'what historical figures would look like', 'true face of', 'true faces of', 'real faces of historical figures', 'true faces of historical figures', 'what historical figures would look like today', 'royalty now', 'roylaty now', 'dama de elche', 'lady of elche', 'lady of elche statue', 'lady of elche reconstruction', 'what did the lady of elche look like', 'spanish artifacts', 'iberian history', 'iberian artifacts', 'spanish history', "dama d'elx", 'lady of elx']


In [63]:
for v in sample_videos:
    t = videos[v]['snippet']['title']
    if t == title:
        print(get_tags(v))

['sleeping with sirens', 'charlotte sands', 'complete collapse', 'let you down']


In [74]:
title = 'Trying School LIFE HACKS to see if they work'
df.sort_values(title, ascending=False)[[title]].head(50)

,Trying School LIFE HACKS to see if they work
Trying School LIFE HACKS to see if they work,1.000000
Who I Do It For,0.778891
GMod Prop Hunt: BIGGEST FAIL YET,0.568073
"My Back Pages_Harrison, Dylan, McGuinn, Clapton, Young, Petty.",0.536356
For Sale: 1860 Antebellum Mansion,0.525726
Entitled People That Are On Another Level - Part 10 - REACTION,0.497397
Sergeant Foley from Modern Warfare 2,0.495851
Nerd³ Tests... N++ Ultimate Edition - Emma Sucks At Platformers,0.488922
CURSED!? Shocking Elimination & Win on Canada Vs The World Already | RuPaul's Drag Race,0.470848
Bolling: Twitter is becoming a fairer platform | The Balance,0.469611
